In [12]:
# Alignments for various species to reference genomes

# loading modules
module load bowtie2
module load minimap2 
module load cd-hit
module load samtools

# checking active 
module list

Currently Loaded Modulefiles:
 1) powerPlant/core    5) Slurm/21.08.8-2      9) bowtie2/2.3.4.3  
 2) texlive/20230914   6) perlbrew/0.76       10) minimap2/2.22    
 3) pandoc/1.19.2      7) perl/5.36.0         11) cd-hit/4.6.1     
 4) git/2.21.0         8) slurm-utils/latest  12) samtools/1.16    

Key:
sticky  


In [15]:
# setting up variables 
# paths
path_raw=/workspace/cfngle/raw-data/
path_results=/workspace/cfngle/results-data/
path_sequences=/workspace/cfngle/results-data/sequences/

# filenames
seq_100bp=("AC_CpG_100bp.fasta" "AS_CpG_100bp.fasta" "EH_CpG_100bp.fasta")

seq_200bp=("AC_CpG_200bp.fasta" "AS_CpG_200bp.fasta" "EH_CpG_200bp.fasta")

seq_500bp=("AC_CpG_500bp.fasta" "AS_CpG_500bp.fasta" "EH_CpG_500bp.fasta")

seq_1000bp=("AC_CpG_1000bp.fasta" "AS_CpG_1000bp.fasta" "EH_CpG_1000bp.fasta")

In [ ]:
#### CD-HIT ####
# clustering sequences with cd-hit
cdhit_opt="-p 1 -c 0.75 -n 4 -T 0 -M 0"
cdhit_nmsfx="_75"

# AC
cd-hit-est-2d -i ${path_results}${seq_200bp[0]} -i2 ${path_results}${seq_200bp[1]} -o ${path_results}cd-hit/AC_AS_cd-hit${cdhit_nmsfx}.fasta $cdhit_opt
cd-hit-est-2d -i ${path_results}${seq_200bp[0]} -i2 ${path_results}${seq_200bp[2]} -o ${path_results}cd-hit/AC_EH_cd-hit${cdhit_nmsfx}.fasta $cdhit_opt
# AS
cd-hit-est-2d -i ${path_results}${seq_200bp[1]} -i2 ${path_results}${seq_200bp[2]} -o ${path_results}cd-hit/AS_EH_cd-hit${cdhit_nmsfx}.fasta $cdhit_opt
cd-hit-est-2d -i ${path_results}${seq_200bp[1]} -i2 ${path_results}${seq_200bp[0]} -o ${path_results}cd-hit/AS_AC_cd-hit${cdhit_nmsfx}.fasta $cdhit_opt
# EH
cd-hit-est-2d -i ${path_results}${seq_200bp[2]} -i2 ${path_results}${seq_200bp[0]} -o ${path_results}cd-hit/EH_AC_cd-hit${cdhit_nmsfx}.fasta $cdhit_opt
cd-hit-est-2d -i ${path_results}${seq_200bp[2]} -i2 ${path_results}${seq_200bp[1]} -o ${path_results}cd-hit/EH_AS_cd-hit${cdhit_nmsfx}.fasta $cdhit_opt


# loop test
: << 'COMMENT'
for i in $seq_200bp; do
    cd-hit-est-2d -i ${path_results}$i -i2 ${path_results}AC -o ${path_results}cd-hit/${i}cd-hit_AC.fasta -c 0.9 -p 1
done
COMMENT

## BOWTIE2 ####

## modules
module load bowtie

## setting up variables 
# paths
path_raw=/workspace/cfngle/raw-data/
path_results=/workspace/cfngle/results-data/bowtie2/
path_sequences=/workspace/cfngle/results-data/sequences/

# filenames
seq_100bp=("AC_CpG_100bp" "AS_CpG_100bp" "EH_CpG_100bp")
seq_200bp=("AC_CpG_200bp" "AS_CpG_200bp" "EH_CpG_200bp")
seq_500bp=("AC_CpG_500bp" "AS_CpG_500bp" "EH_CpG_500bp")
seq_1000bp=("AC_CpG_1000bp" "AS_CpG_1000bp" "EH_CpG_1000bp")

# file ending
suffix=".fasta"

# arguments
bowtie2_args="--very-sensitive --local -p 8"


for (( i=0; i<${#seq_100bp[@]}; i++ )); 
do 
    bowtie2 $bowtie2_args -x ${path_raw}ZF/rgenome/bowtie2-index/ZF_bowtie2 -f -U ${path_sequences}${seq_100bp[$i]}$suffix -S ${path_results}ZF_${seq_100bp[$i]}_bt2_.sam -N 1
    bowtie2 $bowtie2_args -x ${path_raw}ZF/rgenome/bowtie2-index/ZF_bowtie2 -f -U ${path_sequences}${seq_200bp[$i]}$suffix -S ${path_results}ZF_${seq_200bp[$i]}_bt2_.sam -N 1
    bowtie2 $bowtie2_args -x ${path_raw}ZF/rgenome/bowtie2-index/ZF_bowtie2 -f -U ${path_sequences}${seq_500bp[$i]}$suffix -S ${path_results}ZF_${seq_500bp[$i]}_bt2_.sam -N 1
    bowtie2 $bowtie2_args -x ${path_raw}ZF/rgenome/bowtie2-index/ZF_bowtie2 -f -U ${path_sequences}${seq_1000bp[$i]}$suffix -S ${path_results}ZF_${seq_1000bp[$i]}_bt2_.sam -N 1
done

In [42]:
#### MINIMAP2 ####

#ZF
#AC
minimap2 -ax map-ont ${path_raw}ZF/rgenome/GCF_000002035.6_GRCz11_genomic.fna ${path_results}${seq_200bp[0]} > ${path_results}minimap2/ZF_AC_200_minimap.sam
minimap2 -ax map-ont ${path_raw}ZF/rgenome/GCF_000002035.6_GRCz11_genomic.fna ${path_results}${seq_100bp[0]} > ${path_results}minimap2/ZF_AC_100_minimap.sam
#AS
minimap2 -ax map-ont ${path_raw}ZF/rgenome/GCF_000002035.6_GRCz11_genomic.fna ${path_results}${seq_200bp[1]} > ${path_results}minimap2/ZF_AS_200_minimap.sam
minimap2 -ax map-ont ${path_raw}ZF/rgenome/GCF_000002035.6_GRCz11_genomic.fna ${path_results}${seq_100bp[1]} > ${path_results}minimap2/ZF_AS_100_minimap.sam
#EH
minimap2 -ax map-ont ${path_raw}ZF/rgenome/GCF_000002035.6_GRCz11_genomic.fna ${path_results}${seq_200bp[2]} > ${path_results}minimap2/ZF_EH_200_minimap.sam 
minimap2 -ax map-ont ${path_raw}ZF/rgenome/GCF_000002035.6_GRCz11_genomic.fna ${path_results}${seq_100bp[2]} > ${path_results}minimap2/ZF_EH_100_minimap.sam 

path_rgenome="AC/GCF_902167405.1_gadMor3.0_genomic.fasta"
#AC
#AC
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_200bp[0]} > ${path_results}minimap2/AC_AC_200_minimap.sam
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_100bp[0]} > ${path_results}minimap2/AC_AC_100_minimap.sam
#AS
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_200bp[1]} > ${path_results}minimap2/AC_AS_200_minimap.sam
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_100bp[1]} > ${path_results}minimap2/AC_AS_100_minimap.sam
#EH
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_200bp[2]} > ${path_results}minimap2/AC_EH_200_minimap.sam 
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_100bp[2]} > ${path_results}minimap2/AC_EH_100_minimap.sam 

path_rgenome="Chrysophrys_auratus.v.1.0.all.male.map.fasta"
#AS
#AC
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_200bp[0]} > ${path_results}minimap2/AS_AC_200_minimap.sam
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_100bp[0]} > ${path_results}minimap2/AS_AC_100_minimap.sam
#AS
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_200bp[1]} > ${path_results}minimap2/AS_AS_200_minimap.sam
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_100bp[1]} > ${path_results}minimap2/AS_AS_100_minimap.sam
#EH
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_200bp[2]} > ${path_results}minimap2/AS_EH_200_minimap.sam 
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_100bp[2]} > ${path_results}minimap2/AS_EH_100_minimap.sam 

path_rgenome="fMerMel2.1_cnag1.scaffolds.fa"
#EH\
#AC
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_200bp[0]} > ${path_results}minimap2/EH_AC_200_minimap.sam
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_100bp[0]} > ${path_results}minimap2/EH_AC_100_minimap.sam
#AS
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_200bp[1]} > ${path_results}minimap2/EH_AS_200_minimap.sam
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_100bp[1]} > ${path_results}minimap2/EH_AS_100_minimap.sam
#EH
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_200bp[2]} > ${path_results}minimap2/EH_EH_200_minimap.sam 
minimap2 -ax map-ont ${path_raw}path_rgenome ${path_results}${seq_100bp[2]} > ${path_results}minimap2/EH_EH_100_minimap.sam 


In [9]:
## view how many alignments (multiple alignments)

##ZF
#AC
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/ZF_AC_100_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/ZF_AC_200_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/ZF_AC_500_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/ZF_AC_1000_minimap.sam
#AS
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/ZF_AS_100_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/ZF_AS_200_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/ZF_AS_500_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/ZF_AS_1000_minimap.sam
#EH
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/ZF_EH_100_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/ZF_EH_200_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/ZF_EH_500_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/ZF_EH_1000_minimap.sam

##AC
#AC
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AC_AC_100_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AC_AC_200_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AC_AC_500_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AC_AC_1000_minimap.sam
#AS
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AC_AS_100_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AC_AS_200_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AC_AS_500_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AC_AS_1000_minimap.sam
#EH
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AC_EH_100_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AC_EH_200_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AC_EH_500_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AC_EH_1000_minimap.sam

##AS
#AC
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AS_AC_100_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AS_AC_200_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AS_AC_500_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AS_AC_1000_minimap.sam
#AS
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AS_AS_100_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AS_AS_200_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AS_AS_500_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AS_AS_1000_minimap.sam
#EH
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AS_EH_100_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AS_EH_200_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AS_EH_500_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/AS_EH_1000_minimap.sam

##EH
#AC
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/EH_AC_100_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/EH_AC_200_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/EH_AC_500_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/EH_AC_1000_minimap.sam
#AS
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/EH_AS_100_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/EH_AS_200_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/EH_AS_500_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/EH_AS_1000_minimap.sam
#EH
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/EH_EH_100_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/EH_EH_200_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/EH_EH_500_minimap.sam
samtools view -c -F 4 /workspace/cfngle/results-data/minimap2/EH_EH_1000_minimap.sam

173
415
1391
2680
265
497
1081
1975
456
926
2863
5472
16349
15460
14918
14480
1397
2073
3218
4906
10165
12835
18254
25553
844
1231
2149
3650
28688
24797
24642
24530
2261
3698
5642
8460
5126
7118
12052
18420
1588
2508
4191
6812
24022
21442
20341
19491


In [10]:
# view single alignments

##ZF
#AC
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_AC_100_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_AC_200_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_AC_500_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_AC_1000_minimap.sam | wc -l
#AS
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_AS_100_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_AS_200_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_AS_500_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_AS_1000_minimap.sam | wc -l
#EH
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_EH_100_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_EH_200_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_EH_500_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_EH_1000_minimap.sam | wc -l

##AC
#AC
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AC_AC_100_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AC_AC_200_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AC_AC_500_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AC_AC_1000_minimap.sam | wc -l
#AS
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AC_AS_100_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AC_AS_200_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AC_AS_500_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AC_AS_1000_minimap.sam | wc -l
#EH
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AC_EH_100_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AC_EH_200_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AC_EH_500_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AC_EH_1000_minimap.sam | wc -l

##AS
#AC
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AS_AC_100_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AS_AC_200_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AS_AC_500_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AS_AC_1000_minimap.sam | wc -l
#AS
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AS_AS_100_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AS_AS_200_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AS_AS_500_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AS_AS_1000_minimap.sam | wc -l
#EH
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AS_EH_100_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AS_EH_200_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AS_EH_500_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/AS_EH_1000_minimap.sam | wc -l

##EH
#AC
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/EH_AC_100_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/EH_AC_200_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/EH_AC_500_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/EH_AC_1000_minimap.sam | wc -l
#AS
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/EH_AS_100_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/EH_AS_200_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/EH_AS_500_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/EH_AS_1000_minimap.sam | wc -l
#EH
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/EH_EH_100_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/EH_EH_200_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/EH_EH_500_minimap.sam | wc -l
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/EH_EH_1000_minimap.sam | wc -l

107
207
515
887
158
280
543
896
268
430
970
1662
13879
13755
13720
13641
802
1165
1861
2809
6253
7767
10012
12323
514
768
1277
2051
27538
24535
24489
24410
1252
1768
2666
3912
3888
5192
7301
9543
958
1467
2493
3817
19904
18329
18192
18049


In [16]:
minimap2 -ax map-ont ${path_raw}ZF/rgenome/GCF_000002035.6_GRCz11_genomic.fna ${path_sequences}EH_CpG_5000bp.fasta > ${path_results}minimap2/ZF_EH_5000_minimap.sam

[M::mm_idx_gen::32.235*1.63] collected minimizers
[M::mm_idx_gen::40.264*1.89] sorted minimizers
[M::main::40.266*1.89] loaded/built the index for 1923 target sequence(s)
[M::mm_mapopt_update::41.644*1.86] mid_occ = 848
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1923
[M::mm_idx_stat::42.496*1.84] distinct minimizers: 66057654 (43.51% are singletons); average occurrences: 4.867; average spacing: 5.223; total length: 1679203469
[M::worker_pipeline::66.027*2.21] mapped 16751 sequences
[M::main] Version: 2.22-r1101
[M::main] CMD: minimap2 -ax map-ont /workspace/cfngle/raw-data/ZF/rgenome/GCF_000002035.6_GRCz11_genomic.fna /workspace/cfngle/results-data/sequences/EH_CpG_5000bp.fasta
[M::main] Real time: 66.285 sec; CPU: 146.265 sec; Peak RSS: 6.608 GB


In [17]:
samtools view -F 0x904 /workspace/cfngle/results-data/minimap2/ZF_EH_5000_minimap.sam | wc -l

5557


In [ ]:
#### MAFFT ####

## modules
module load mafft

## setting up variables 

# paths
path_raw=/workspace/cfngle/raw-data/
path_results=/workspace/cfngle/results-data/mafft/
path_sequences=/workspace/cfngle/results-data/sequences/

# arguments
mafft_args="--auto --thread -1"

mafft $mafft_args ${path_sequences}AC_AS_EH_CpG_100bp.fasta > ${path_results}AC_AS_EH_CpG_100bp.fasta 
mafft $mafft_args ${path_sequences}AC_AS_EH_CpG_200bp.fasta > ${path_results}AC_AS_EH_CpG_200bp.fasta 
mafft $mafft_args ${path_sequences}AC_AS_EH_CpG_500bp.fasta > ${path_results}AC_AS_EH_CpG_500bp.fasta 
mafft $mafft_args ${path_sequences}AC_AS_EH_CpG_1000bp.fasta > ${path_results}AC_AS_EH_CpG_1000bp.fasta 